In [1]:
import pyspark.sql.functions as f

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7426,application_1589299642358_1922,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure
{"conf": {
    "spark.app.name": "datavirus_timetable"
}}

A session has already been started. If you intend to recreate the session with new configurations, please include the -f argument.


In [3]:
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7272,application_1589299642358_1768,pyspark,idle,Link,Link,
7292,application_1589299642358_1788,pyspark,idle,Link,Link,
7315,application_1589299642358_1811,pyspark,idle,Link,Link,
7326,application_1589299642358_1822,pyspark,idle,Link,Link,
7331,application_1589299642358_1827,pyspark,busy,Link,Link,
7337,application_1589299642358_1833,pyspark,idle,Link,Link,
7347,application_1589299642358_1843,pyspark,dead,Link,Link,
7357,application_1589299642358_1853,pyspark,idle,Link,Link,
7369,application_1589299642358_1865,pyspark,busy,Link,Link,
7371,application_1589299642358_1867,pyspark,idle,Link,Link,


In [26]:
stop_time = spark.read.orc("/data/sbb/timetables/orc/stop_times/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
stop_time.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trip_id: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- stop_sequence: short (nullable = true)
 |-- pickup_type: byte (nullable = true)
 |-- drop_off_type: byte (nullable = true)

In [6]:
stop_time.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|pickup_type|drop_off_type|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|1.TA.1-1-B-j19-1.1.R|    04:20:00|      04:20:00|8500010:0:3|            1|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:24:00|      04:24:00|8500020:0:3|            2|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:28:00|      04:28:00|8500021:0:5|            3|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:30:00|      04:30:00|8517131:0:2|            4|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:32:00|      04:32:00|8500300:0:5|            5|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:35:00|      04:35:00|8500313:0:2|            6|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:37:00|      04:38:00|8500301:0:3|           

In [12]:
@f.udf
def parent_station_id(stop_id):
    return stop_id.split(':')[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
stop_time = (
    stop_time
    .select(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])
    .withColumn('stop_id', parent_station_id(stop_time.stop_id))
    
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
stop_time.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|
+--------------------+------------+--------------+-------+-------------+
|1.TA.1-1-B-j19-1.1.R|    04:20:00|      04:20:00|8500010|            1|
|1.TA.1-1-B-j19-1.1.R|    04:24:00|      04:24:00|8500020|            2|
|1.TA.1-1-B-j19-1.1.R|    04:28:00|      04:28:00|8500021|            3|
|1.TA.1-1-B-j19-1.1.R|    04:30:00|      04:30:00|8517131|            4|
|1.TA.1-1-B-j19-1.1.R|    04:32:00|      04:32:00|8500300|            5|
|1.TA.1-1-B-j19-1.1.R|    04:35:00|      04:35:00|8500313|            6|
|1.TA.1-1-B-j19-1.1.R|    04:37:00|      04:38:00|8500301|            7|
|1.TA.1-1-B-j19-1.1.R|    04:40:00|      04:41:00|8500302|            8|
|1.TA.1-1-B-j19-1.1.R|    04:45:00|      04:45:00|8500303|            9|
|1.TA.1-1-B-j19-1.1.R|    04:48:00|      04:49:00|8500320|           10|
+--------------------+------------+--------------+-

In [39]:
departure = stop_time.alias('departure')
arrival = stop_time.withColumn('stop_sequence', stop_time.stop_sequence - 1).alias('arrival')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
connection = (
    departure
    .join(arrival, ['trip_id', 'stop_sequence'])
    .select(
        departure.trip_id,
        departure.stop_sequence,
        departure.stop_id.alias('start_id'),
        departure.departure_time.alias('start_time'),
        arrival.arrival_time.alias('stop_time'),
        arrival.stop_id.alias('stop_id')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
connection.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o381.showString.
: java.util.concurrent.TimeoutException: Futures timed out after [300 seconds]
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:219)
	at scala.concurrent.impl.Promise$DefaultPromise.result(Promise.scala:223)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:201)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:136)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:367)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.e